In [30]:
#importing dependencies
import pandas as pd
import os

# set the path for the files
school_data_to_load = os.path.join('Resources','schools_complete.csv')
student_data_to_load = os.path.join('Resources', 'students_complete.csv')

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [29]:
#getting the values for the first data frame
total_schools = school_data_complete['school_name'].nunique()
total_students = school_data_complete['student_name'].count()
total_budget = school_data['budget'].sum()
avg_math_score = round(school_data_complete ['math_score'].mean(),2)
avg_reading_score = round(school_data_complete['reading_score'].mean(),2)

#find the kids that are passing
#math
passing_math = school_data_complete[school_data_complete['math_score'] >= 70]
passing_math_counts = passing_math['math_score'].count()
#reading
passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70]
passing_reading_counts = passing_reading['reading_score'].count()
#both
passing_both = school_data_complete[(school_data_complete['math_score'] >= 70) & 
                                    (school_data_complete['reading_score'] >= 70)]
passing_both_counts = passing_both['student_name'].count()

#calculations for the percentages
passing_perentage_math = round((int(passing_math_counts)/total_students)*100,2)
passing_percentage_reading = round((int(passing_reading_counts)/total_students)*100,2)
passing_percentage_both = round((int(passing_both_counts)/total_students)*100,2)

#creating the data frame
#dictionaries
summary_dictionaries = [{'Total Schools': total_schools, 'Total Students': total_students, 
                        'Total Budget': total_budget, 'Avg Math Score': avg_math_score,
                       'Avg Reading Score': avg_reading_score, '% Passing Math': passing_perentage_math,
                        '% Passing Reading': passing_percentage_reading, '% Overall Passing': passing_percentage_both}]
#makin the frame
summary_df = pd.DataFrame(summary_dictionaries)
summary_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,74.98,85.81,65.17
